In [1]:
# install Python packages used in this notebook
!pip install pandas numpy python-geohash geohash2 folium

In [2]:
import os
import time
import shutil
import zipfile
import geohash
import geohash2
import numpy as np
import pandas as pd
import folium
import multiprocessing

from io import BytesIO
from urllib.request import urlopen
from folium.features import DivIcon

pd.set_option('max_rows', 15)

# download the current MaxMind package
maxmindURL = 'http://geolite.maxmind.com/download/geoip/database/GeoLite2-City-CSV.zip'
with urlopen(maxmindURL) as response:
    with zipfile.ZipFile(BytesIO(response.read())) as file:
        file.extractall()

# find the newest directory, in case there are old directories left over from previous runs
maxmindDirectory = sorted( [ f for f in os.listdir() if os.path.isdir(f) and f.startswith('GeoLite2-City-CSV') ] )[-1]

# load the MaxMind network data 
maxmindNetworks = pd.read_csv( maxmindDirectory + '/GeoLite2-City-Blocks-IPv4.csv', 
                               header=0, 
                               usecols=['network','geoname_id','latitude','longitude'], 
                               dtype=str,
                               na_values='',
                               keep_default_na=False )
maxmindNetworks = maxmindNetworks.dropna()
maxmindNetworks['geoname_id'] = maxmindNetworks['geoname_id'].astype(int)
maxmindNetworks['latitude'] = maxmindNetworks['latitude'].astype(float)
maxmindNetworks['longitude'] = maxmindNetworks['longitude'].astype(float)
maxmindNetworks.to_csv('maxmindNetworks.csv', index=False)

# load the MaxMind location data 
maxmindLocations = pd.read_csv( maxmindDirectory + '/GeoLite2-City-Locations-en.csv', 
                                header=0, 
                                usecols=['geoname_id','country_iso_code','country_name','subdivision_1_iso_code','subdivision_1_name','city_name'], 
                                dtype=str,
                                na_values='',
                                keep_default_na=False )
maxmindLocations = maxmindLocations.dropna(subset=['country_iso_code'])
maxmindLocations['geoname_id'] = maxmindLocations['geoname_id'].astype(int)
for column in ['country_name','subdivision_1_name','city_name']:
    maxmindLocations[column] = maxmindLocations[column].apply(lambda name: name.replace(',', '') if type(name) is str else name)
maxmindLocations.to_csv('maxmindLocations.csv', index=False)

In [3]:
# display MaxMind network data
maxmindNetworks

,network,geoname_id,latitude,longitude
0,1.0.0.0/24,2151718,-37.7000,145.1833
1,1.0.1.0/24,1810821,26.0614,119.3061
2,1.0.2.0/23,1810821,26.0614,119.3061
3,1.0.4.0/22,2077456,-33.4940,143.2104
4,1.0.8.0/21,1809858,23.1167,113.2500
5,1.0.16.0/20,1850147,35.6850,139.7514
6,1.0.32.0/19,1809858,23.1167,113.2500
...,...,...,...,...
2662725,223.255.236.0/22,1796236,31.0456,121.3997
2662726,223.255.240.0/22,1819730,22.2500,114.1667


In [4]:
# display MaxMind location data
maxmindLocations.sort_values('geoname_id')

,geoname_id,country_iso_code,country_name,subdivision_1_iso_code,subdivision_1_name,city_name
0,18918,CY,Cyprus,04,Ammochostos,Protaras
1,49518,RW,Rwanda,NaN,NaN,NaN
2,49747,SO,Somalia,BK,Bakool,Oddur
3,51537,SO,Somalia,NaN,NaN,NaN
4,53654,SO,Somalia,BN,Banaadir,Mogadishu
5,54225,SO,Somalia,SH,Lower Shabeelle,Merca
6,55671,SO,Somalia,JH,Lower Juba,Kismayo
...,...,...,...,...,...,...
103001,11789760,CA,Canada,NB,New Brunswick,Shippagan
103002,11790338,CH,Switzerland,VD,Vaud,Servion


In [5]:
# add geohash of latitude/longitude and display network data again

def parallelApply(frame, function):
    with multiprocessing.Pool(os.cpu_count()) as pool:
        result = pd.concat(pool.map(function, np.array_split(frame, os.cpu_count())))
    return result

def calculateGeohash6(frame):
    return frame.apply(lambda row: geohash2.encode(row['latitude'],row['longitude'],precision=6),axis=1)

before = time.time()
#maxmindNetworks['geohash6'] = maxmindNetworks.apply(lambda row: geohash2.encode(row['latitude'],row['longitude'],precision=6),axis=1)
maxmindNetworks['geohash6'] = parallelApply(maxmindNetworks, calculateGeohash6)
elapsed = time.time() - before
print('elapsed time: ' + str(elapsed) + ' seconds')
maxmindNetworks

elapsed time: 28.43257474899292 seconds


,network,geoname_id,latitude,longitude,geohash6
0,1.0.0.0/24,2151718,-37.7000,145.1833,r1r1x8
1,1.0.1.0/24,1810821,26.0614,119.3061,wssu6b
2,1.0.2.0/23,1810821,26.0614,119.3061,wssu6b
3,1.0.4.0/22,2077456,-33.4940,143.2104,r4jc6y
4,1.0.8.0/21,1809858,23.1167,113.2500,ws0e90
5,1.0.16.0/20,1850147,35.6850,139.7514,xn77h0
6,1.0.32.0/19,1809858,23.1167,113.2500,ws0e90
...,...,...,...,...,...
2662725,223.255.236.0/22,1796236,31.0456,121.3997,wtw2de
2662726,223.255.240.0/22,1819730,22.2500,114.1667,wecntf


In [6]:
# merge network and location data and display results
maxmindNetworkLocations = maxmindNetworks.join(maxmindLocations.set_index('geoname_id'), on='geoname_id') 
maxmindNetworkLocations

,network,geoname_id,latitude,longitude,geohash6,country_iso_code,country_name,subdivision_1_iso_code,subdivision_1_name,city_name
0,1.0.0.0/24,2151718,-37.7000,145.1833,r1r1x8,AU,Australia,VIC,Victoria,Research
1,1.0.1.0/24,1810821,26.0614,119.3061,wssu6b,CN,China,FJ,Fujian,Fuzhou
2,1.0.2.0/23,1810821,26.0614,119.3061,wssu6b,CN,China,FJ,Fujian,Fuzhou
3,1.0.4.0/22,2077456,-33.4940,143.2104,r4jc6y,AU,Australia,NaN,NaN,NaN
4,1.0.8.0/21,1809858,23.1167,113.2500,ws0e90,CN,China,GD,Guangdong,Guangzhou
5,1.0.16.0/20,1850147,35.6850,139.7514,xn77h0,JP,Japan,13,Tokyo,Tokyo
6,1.0.32.0/19,1809858,23.1167,113.2500,ws0e90,CN,China,GD,Guangdong,Guangzhou
...,...,...,...,...,...,...,...,...,...,...
2662725,223.255.236.0/22,1796236,31.0456,121.3997,wtw2de,CN,China,SH,Shanghai,Shanghai
2662726,223.255.240.0/22,1819730,22.2500,114.1667,wecntf,HK,Hong Kong,NaN,NaN,NaN


In [7]:
# group networks by location
maxmindNetworksGroupedByLocation = maxmindNetworkLocations.groupby('geoname_id')

In [8]:
# calculate average coordinates and geohash of networks in each location

def parallelApplyGrouped(groupedData, function):
    with multiprocessing.Pool(os.cpu_count()) as pool:
        result = pool.map( function, [ [name,group] for name,group in groupedData ] )
    return result

def networkGroupSummary(args):
    geoname_id = args[0]
    data = args[1]
    networkCount = len(data)
    coordinateCount = len(data.groupby('geohash6').count())
    averageLatitude = data['latitude'].mean()
    averageLongitude = data['longitude'].mean()
    geohash6 = geohash2.encode(averageLatitude,averageLongitude,precision=6)
    return [geoname_id,networkCount,coordinateCount,averageLatitude,averageLongitude,geohash6]
    
before = time.time()
columns = ['geoname_id','networkCount','coordinateCount','averageLatitude','averageLongitude','geohash6']
#maxmindNetworkGroupSummary = pd.DataFrame( [ networkGroupSummary(g) for g in list(maxmindNetworksGroupedByLocation.groups.keys()) ], columns=columns )
maxmindNetworkGroupSummary = pd.DataFrame( parallelApplyGrouped(maxmindNetworksGroupedByLocation, networkGroupSummary), columns=columns )
elapsed = time.time() - before
print('elapsed time: ' + str(elapsed) + ' seconds')
maxmindNetworkGroupSummary

elapsed time: 58.46319007873535 seconds


,geoname_id,networkCount,coordinateCount,averageLatitude,averageLongitude,geohash6
0,18918,2,1,35.0125,34.0583,sy0ppj
1,49518,62,1,-2.0000,30.0000,kxtkde
2,49747,1,1,4.1213,43.8895,sbxpqw
3,51537,30,1,10.0000,49.0000,t1fbgh
4,53654,4,1,2.0462,45.3341,t025xs
5,54225,1,1,1.7159,44.7717,sbrcfb
6,55671,1,1,-0.3582,42.5454,kzymbn
...,...,...,...,...,...,...
97665,11789760,4,1,47.7287,-64.7247,f83zx0
97666,11790338,1,1,46.5710,6.7783,u0kbfj


In [9]:
# add location data to network group summary
maxmindLocationSummary = maxmindNetworkGroupSummary.join(maxmindLocations.set_index('geoname_id'), on='geoname_id')
maxmindLocationSummary = maxmindLocationSummary.dropna(subset=['country_iso_code'])
maxmindLocationSummary

,geoname_id,networkCount,coordinateCount,averageLatitude,averageLongitude,geohash6,country_iso_code,country_name,subdivision_1_iso_code,subdivision_1_name,city_name
0,18918,2,1,35.0125,34.0583,sy0ppj,CY,Cyprus,04,Ammochostos,Protaras
1,49518,62,1,-2.0000,30.0000,kxtkde,RW,Rwanda,NaN,NaN,NaN
2,49747,1,1,4.1213,43.8895,sbxpqw,SO,Somalia,BK,Bakool,Oddur
3,51537,30,1,10.0000,49.0000,t1fbgh,SO,Somalia,NaN,NaN,NaN
4,53654,4,1,2.0462,45.3341,t025xs,SO,Somalia,BN,Banaadir,Mogadishu
5,54225,1,1,1.7159,44.7717,sbrcfb,SO,Somalia,SH,Lower Shabeelle,Merca
6,55671,1,1,-0.3582,42.5454,kzymbn,SO,Somalia,JH,Lower Juba,Kismayo
...,...,...,...,...,...,...,...,...,...,...,...
97665,11789760,4,1,47.7287,-64.7247,f83zx0,CA,Canada,NB,New Brunswick,Shippagan
97666,11790338,1,1,46.5710,6.7783,u0kbfj,CH,Switzerland,VD,Vaud,Servion


In [10]:
# display summary for locations with lots of networks and coordinates
maxmindLocationSummary.sort_values(['coordinateCount','networkCount'],ascending=False)

,geoname_id,networkCount,coordinateCount,averageLatitude,averageLongitude,geohash6,country_iso_code,country_name,subdivision_1_iso_code,subdivision_1_name,city_name
5081,756135,1984,200,52.243812,21.003704,u3qcjy,PL,Poland,MZ,Mazovia,Warsaw
35535,2950159,5213,193,52.513503,13.403651,u33d9p,DE,Germany,BE,Land Berlin,Berlin
66440,3530597,4232,162,19.431386,-99.138600,9g3w81,MX,Mexico,CMX,Mexico City,Mexico City
4873,745044,3548,123,41.020238,28.976520,sxk977,TR,Turkey,34,Istanbul,Istanbul
91300,6167865,9858,101,43.668768,-79.428137,dpz82t,CA,Canada,ON,Ontario,Toronto
14997,2193733,4105,99,-36.869042,174.767407,rckq2b,NZ,New Zealand,AUK,Auckland,Auckland
76115,4699066,7093,98,29.778251,-95.445610,9vk1kh,US,United States,TX,Texas,Houston
...,...,...,...,...,...,...,...,...,...,...,...
97651,11748289,1,1,-26.925500,152.109100,r7hnv6,AU,Australia,QLD,Queensland,Cherry Creek
97655,11778479,1,1,35.616700,139.633300,xn7697,JP,Japan,13,Tokyo,Tamagawa


In [11]:
# this function plots the networks in a location on a map

def drawmap(geoname_id,zoomLevel):

    countryName = maxmindLocations.loc[maxmindLocations['geoname_id']==geoname_id,'country_name'].item()
    subdivisionName = maxmindLocations.loc[maxmindLocations['geoname_id']==geoname_id,'subdivision_1_name'].item()
    cityName = maxmindLocations.loc[maxmindLocations['geoname_id']==geoname_id,'city_name'].item()

    group = maxmindNetworksGroupedByLocation.get_group(geoname_id)
    networkCount = len(group)
    averageLatitude, averageLongitude = group[['latitude','longitude']].mean()

    map = folium.Map(location=[averageLatitude, averageLongitude], zoom_start=zoomLevel)

    points = group.groupby(['latitude','longitude'])
    coordinateCount = len(list(points.groups.keys()))

    for key, group in points:
        latitude,longitude = key
        folium.features.Circle(location=[latitude, longitude], radius=200, color='blue').add_to(map)

    print('location ' + str(geoname_id) + ': ' + str(networkCount) + ' networks at ' + str(coordinateCount) + ' coordinates in ' + cityName + ', ' + subdivisionName + ', ' + countryName)
    return map

In [12]:
drawmap(3114472,11) # Pamplona

location 3114472: 228 networks at 6 coordinates in Pamplona, Navarre, Spain


In [13]:
drawmap(4335045,12) # New Orleans

location 4335045: 818 networks at 23 coordinates in New Orleans, Louisiana, United States


In [14]:
drawmap(2867714,11) # Munich

location 2867714: 2273 networks at 67 coordinates in Munich, Bavaria, Germany


In [15]:
drawmap(2950159,10) # Berlin

location 2950159: 5213 networks at 197 coordinates in Berlin, Land Berlin, Germany


In [16]:
# this function finds a geohah that encloses all of the networks in a location and plots them on a map

def findGeohashBBox(minLatitude, minLongitude, maxLatitude, maxLongitude):

    for p in range(12,0,-1):
        geohashCode = geohash2.encode( (maxLatitude+minLatitude)/2, (maxLongitude+minLongitude)/2, precision=p)
        geohashBBox = geohash.bbox(geohashCode)    
        if minLatitude < geohashBBox['s']: continue
        if maxLatitude > geohashBBox['n']: continue
        if minLongitude < geohashBBox['w']: continue
        if maxLongitude > geohashBBox['e']: continue
        return (geohashCode, geohashBBox)
    print('no geohash found that encloses latitude ' + str(minLatitude) + ' to ' + str(maxLatitude) + ', longitude ' + str(minLongitude) + ' to ' + str(maxLongitude))
    return (None, None)

def drawmapWithGeohashBBox(geoname_id,zoomLevel):

    countryName = maxmindLocations.loc[maxmindLocations['geoname_id']==geoname_id,'country_name'].item()
    subdivisionName = maxmindLocations.loc[maxmindLocations['geoname_id']==geoname_id,'subdivision_1_name'].item()
    cityName = maxmindLocations.loc[maxmindLocations['geoname_id']==geoname_id,'city_name'].item()

    group = maxmindNetworksGroupedByLocation.get_group(geoname_id)
    networkCount = len(group)
    minLatitude, minLongitude = group[['latitude','longitude']].min()
    avgLatitude, avgLongitude = group[['latitude','longitude']].mean()
    maxLatitude, maxLongitude = group[['latitude','longitude']].max()

    map = folium.Map(location=[avgLatitude, avgLongitude], zoom_start=zoomLevel)

    points = group.groupby(['latitude','longitude'])
    coordinateCount = len(list(points.groups.keys()))

    for key, group in points:
        latitude,longitude = key
        folium.features.Circle(location=[latitude, longitude], radius=200, color='blue').add_to(map)

    (geohashCode, geohashBBox) = findGeohashBBox(minLatitude, minLongitude, maxLatitude, maxLongitude)
    if geohashCode is None: return None
    
    folium.features.RectangleMarker([(geohashBBox['s'], geohashBBox['w']),(geohashBBox['n'], geohashBBox['e'])],weight=0, fill_color='blue', fill_opacity=0.2,).add_to(map)
        
    print('geohash "' + geohashCode + '" for location ' + str(geoname_id) + ': ' + str(networkCount) + ' networks at ' + str(coordinateCount) + ' coordinates in ' + cityName + ', ' + subdivisionName + ', ' + countryName)
    return map

In [17]:
drawmapWithGeohashBBox(3114472,11) # Pamplona

geohash "ezwg" for location 3114472: 228 networks at 6 coordinates in Pamplona, Navarre, Spain


In [18]:
drawmapWithGeohashBBox(4335045,12) # New Orleans

no geohash found that encloses latitude 29.9049 to 30.0801, longitude -90.2057 to -89.879


In [19]:
drawmapWithGeohashBBox(2867714,8) # Munich

geohash "u28" for location 2867714: 2273 networks at 67 coordinates in Munich, Bavaria, Germany


In [20]:
drawmapWithGeohashBBox(2950159,8) # Berlin

geohash "u33" for location 2950159: 5213 networks at 197 coordinates in Berlin, Land Berlin, Germany


In [21]:
# this function finds a geohah that contains the centroid of the networks in a location and plots them on a map

def drawmapWithGeohashCentroid(geoname_id,zoomLevel):

    countryName = maxmindLocations.loc[maxmindLocations['geoname_id']==geoname_id,'country_name'].item()
    subdivisionName = maxmindLocations.loc[maxmindLocations['geoname_id']==geoname_id,'subdivision_1_name'].item()
    cityName = maxmindLocations.loc[maxmindLocations['geoname_id']==geoname_id,'city_name'].item()

    group = maxmindNetworksGroupedByLocation.get_group(geoname_id)
    networkCount = len(group)

    avgLatitude, avgLongitude = group[['latitude','longitude']].mean()
    map = folium.Map(location=[avgLatitude, avgLongitude], zoom_start=zoomLevel)

    points = group.groupby(['latitude','longitude'])
    coordinateCount = len(list(points.groups.keys()))

    for key, group in points:
        latitude,longitude = key
        folium.features.Circle(location=[latitude, longitude], radius=200, color='blue').add_to(map)

    geohashCode = geohash2.encode(avgLatitude, avgLongitude, precision=5)
    geohashBBox = geohash.bbox(geohashCode)    
    folium.features.RectangleMarker([(geohashBBox['s'], geohashBBox['w']),(geohashBBox['n'], geohashBBox['e'])],weight=0, fill_color='blue', fill_opacity=.33).add_to(map)
        
    print('geohash "' + geohashCode + '" for location ' + str(geoname_id) + ': ' + str(networkCount) + ' networks at ' + str(coordinateCount) + ' coordinates in ' + cityName + ', ' + subdivisionName + ', ' + countryName)
    return map

In [22]:
drawmapWithGeohashCentroid(3114472,11) # Pamplona

geohash "ezwgd" for location 3114472: 228 networks at 6 coordinates in Pamplona, Navarre, Spain


In [23]:
drawmapWithGeohashCentroid(4335045,12) # New Orleans

geohash "9vrfq" for location 4335045: 818 networks at 23 coordinates in New Orleans, Louisiana, United States


In [24]:
drawmapWithGeohashCentroid(2867714,11) # Munich

geohash "u281z" for location 2867714: 2273 networks at 67 coordinates in Munich, Bavaria, Germany


In [25]:
drawmapWithGeohashCentroid(2950159,10) # Berlin

geohash "u33d9" for location 2950159: 5213 networks at 197 coordinates in Berlin, Land Berlin, Germany


In [26]:
# this function finds a geohah that contains the centroid of the networks in a location and plots them on a map

def drawmapWithGeohash5and6Centroid(geoname_id,zoomLevel):

    countryName = maxmindLocations.loc[maxmindLocations['geoname_id']==geoname_id,'country_name'].item()
    subdivisionName = maxmindLocations.loc[maxmindLocations['geoname_id']==geoname_id,'subdivision_1_name'].item()
    cityName = maxmindLocations.loc[maxmindLocations['geoname_id']==geoname_id,'city_name'].item()

    group = maxmindNetworksGroupedByLocation.get_group(geoname_id)
    networkCount = len(group)

    avgLatitude, avgLongitude = group[['latitude','longitude']].mean()
    map = folium.Map(location=[avgLatitude, avgLongitude], zoom_start=zoomLevel)

    points = group.groupby(['latitude','longitude'])
    coordinateCount = len(list(points.groups.keys()))

    for key, group in points:
        latitude,longitude = key
        folium.features.Circle(location=[latitude, longitude], radius=200, color='blue').add_to(map)

    geohashCode5 = geohash2.encode(avgLatitude, avgLongitude, precision=5)
    geohashBBox5 = geohash.bbox(geohashCode5)    
    folium.features.RectangleMarker([(geohashBBox5['s'], geohashBBox5['w']),(geohashBBox5['n'], geohashBBox5['e'])],weight=0, fill_color='blue', fill_opacity=.33).add_to(map)
        
    geohashCode6 = geohash2.encode(avgLatitude, avgLongitude, precision=6)
    geohashBBox6 = geohash.bbox(geohashCode6)    
    folium.features.RectangleMarker([(geohashBBox6['s'], geohashBBox6['w']),(geohashBBox6['n'], geohashBBox6['e'])],weight=0, fill_color='blue', fill_opacity=.5).add_to(map)

    titleCoordinates = [geohashBBox5['s'], (geohashBBox5['w']+geohashBBox5['e'])/2]
    
    titleHTML = cityName + ', ' + subdivisionName + ', ' + countryName + '<br>' + \
                'location: ' + str(geoname_id) + '<br>' + \
                'geohashes: ' + geohashCode5 + ' and ' + geohashCode6 + '<br>' + \
                str(networkCount) + ' networks at ' + str(coordinateCount) + ' coordinates'

    titleCSS = 'color: blue;' + \
                'font-size: 9pt;' + \
                'font-weight: bold;' + \
                'font-style: italic;' + \
                'text-align: center; ' + \
                'padding: 5px;' + \
                'background-color: rgba(255,255,255,0.6);' + \
                'position: fixed;' + \
                'transform: translateX(-50%);'

    folium.map.Marker(
        titleCoordinates,
        icon=DivIcon(icon_size=(300,100), 
                     icon_anchor=(0,0), 
                     html='<div style="' + titleCSS + '">' + titleHTML + '</div>')).add_to(map)
    
    #print('geohashes "' + geohashCode5 + '" and "' + geohashCode6 + '" for location ' + str(geoname_id) + ': ' + str(networkCount) + ' networks at ' + str(coordinateCount) + ' coordinates in ' + cityName + ', ' + subdivisionName + ', ' + countryName)
    return map

#drawmapWithGeohash5and6Centroid(3114472,11) # Pamplona

In [27]:
drawmapWithGeohash5and6Centroid(3114472,11) # Pamplona

In [28]:
drawmapWithGeohash5and6Centroid(4335045,12) # New Orleans

In [29]:
drawmapWithGeohash5and6Centroid(2867714,11) # Munich

In [30]:
drawmapWithGeohash5and6Centroid(2950159,10) # Berlin

In [40]:
drawmapWithGeohash5and6Centroid(5043473,11) # Rochester

In [31]:
drawmapWithGeohash5and6Centroid(3114472,11).save('map.Pamplona.html')
drawmapWithGeohash5and6Centroid(4335045,12).save('map.NewOrleans.html') 
drawmapWithGeohash5and6Centroid(2867714,11).save('map.Munich.html') 
drawmapWithGeohash5and6Centroid(2950159,10).save('map.Berlin.html') 

In [32]:
# store a sorted list of all country codes and names
countryCodes = maxmindLocations[['country_iso_code','country_name']].drop_duplicates().sort_values('country_iso_code')
countryCodes.to_csv('countryCodes.all.csv',index=False)
countryCodes

,country_iso_code,country_name
56038,AD,Andorra
624,AE,United Arab Emirates
6658,AF,Afghanistan
71548,AG,Antigua and Barbuda
71467,AI,Anguilla
1255,AL,Albania
197,AM,Armenia
...,...,...
73986,WS,Samoa
5917,XK,Kosovo


In [33]:
# store a sorted list of all subdivison codes and names
subdivisionCodesAll = maxmindLocations[['country_iso_code','subdivision_1_iso_code','country_name','subdivision_1_name']].drop_duplicates(['country_iso_code','subdivision_1_iso_code']).sort_values(['country_iso_code','subdivision_1_iso_code'])
subdivisionCodesAll.to_csv('subdivisionCodes.all.csv',index=False)
subdivisionCodesAll

,country_iso_code,subdivision_1_iso_code,country_name,subdivision_1_name
56046,AD,02,Andorra,Canillo
56045,AD,03,Andorra,Encamp
56043,AD,04,Andorra,La Massana
56038,AD,05,Andorra,Ordino
56039,AD,06,Andorra,Sant Julià de Loria
56040,AD,07,Andorra,Andorra la Vella
56042,AD,08,Andorra,Escaldes-Engordany
...,...,...,...,...
6134,ZW,BU,Zimbabwe,Bulawayo
6131,ZW,HA,Zimbabwe,Harare


In [34]:
# this function returns a list of subdivision codes and names for the specified country

def subdivisionCodes(countryCode):
    subdivisions = maxmindLocations[maxmindLocations['country_iso_code']==countryCode][['subdivision_1_iso_code','subdivision_1_name']].drop_duplicates()
    return subdivisions

In [35]:
# get a sorted list of subdivison codes and names for the U.S.
subdivisionCodesUS = subdivisionCodes('US').sort_values('subdivision_1_iso_code')
subdivisionCodesUS.to_csv('subdivisionCodes.US.csv',index=False)
subdivisionCodesUS

,subdivision_1_iso_code,subdivision_1_name
92558,AK,Alaska
74017,AL,Alabama
74032,AR,Arkansas
90815,AZ,Arizona
90819,CA,California
91851,CO,Colorado
82461,CT,Connecticut
...,...,...
74025,VA,Virginia
90002,VT,Vermont


In [36]:
# get a sorted list of subdivison codes and names for Russia
subdivisionCodesRU = subdivisionCodes('RU').sort_values('subdivision_1_iso_code')
subdivisionCodesRU.to_csv('subdivisionCodes.RU.csv',index=False)
subdivisionCodesRU

,subdivision_1_iso_code,subdivision_1_name
1480,AD,Respublika Adygeya
8366,AL,Altai
8340,ALT,Altai Krai
12413,AMU,Amurskaya Oblast'
1529,ARK,Arkhangelskaya
1968,AST,Astrakhanskaya Oblast'
1476,BA,Bashkortostan
...,...,...
1481,VLG,Vologodskaya Oblast'
1495,VOR,Voronezhskaya Oblast'


In [37]:
# get a sorted list of subdivison codes and names for China
subdivisionCodesCN = subdivisionCodes('CN').sort_values('subdivision_1_iso_code')
subdivisionCodesCN.to_csv('subdivisionCodes.CN.csv',index=False)
subdivisionCodesCN

,subdivision_1_iso_code,subdivision_1_name
8716,AH,Anhui
10455,BJ,Beijing
10371,CQ,Chongqing
10342,FJ,Fujian
10339,GD,Guangdong
10343,GS,Gansu
10361,GX,Guangxi
...,...,...
10337,SX,Shanxi
10392,TJ,Tianjin


In [38]:
# store a sorted list of subdivison codes and names actually used by networks
subdivisionCodesUsed = maxmindLocationSummary[['country_iso_code','subdivision_1_iso_code','country_name','subdivision_1_name']].drop_duplicates(['country_iso_code','subdivision_1_iso_code']).sort_values(['country_iso_code','subdivision_1_iso_code'])
subdivisionCodesUsed.to_csv('subdivisionCodes.used.csv',index=False)
subdivisionCodesUsed

,country_iso_code,subdivision_1_iso_code,country_name,subdivision_1_name
51600,AD,02,Andorra,Canillo
51599,AD,03,Andorra,Encamp
51597,AD,04,Andorra,La Massana
51592,AD,05,Andorra,Ordino
51593,AD,06,Andorra,Sant Julià de Loria
51594,AD,07,Andorra,Andorra la Vella
51596,AD,08,Andorra,Escaldes-Engordany
...,...,...,...,...
5964,ZW,BU,Zimbabwe,Bulawayo
5961,ZW,HA,Zimbabwe,Harare
